In [2]:
import pandas as pd
from ufal.morphodita import *
import csv
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
data = pd.read_csv('articles_cleaned.csv',  sep='|')
data.columns = ['id','query','url', 'headline', 'paragraphs']
data['paragraphs'] = data['paragraphs'].fillna('')

In [6]:
data =  data[data['paragraphs'] != '\xa0']  

In [7]:
lemma_column = data.shape[0]*[[]]

In [8]:
data['lemmas'] = lemma_column

In [11]:
tagger = Tagger.load('../../morphodita/czech-morfflex-pdt-161115.tagger')
forms = Forms()

In [12]:
for index, row in data.iterrows():
    paragraph = row['paragraphs']
    lemmas = TaggedLemmas()
    tokens = TokenRanges()
    tokenizer = tagger.newTokenizer()
    tokenizer.setText(paragraph)
    allLemmas = []
    while tokenizer.nextSentence(forms, tokens):
        newLemmas = []
        tagger.tag(forms, lemmas)
        newLemmas = [l.lemma for l in lemmas]
        allLemmas.extend(newLemmas)
#     print(allLemmas)
    row['lemmas'] = allLemmas

In [13]:
lemma_data = data['lemmas']

In [14]:
lemma_data

0       [souhrn, meteorologický, informace, uplynulý_^...
1       [kolem-1, rozsáhlý, tlakový, výše_^(velikost_a...
2       [počet, oběť, bouřlivý, počasí, a-1, záplava, ...
3       [studený, fronta, se_^(zvr._zájmeno/částice), ...
4       [událost_,a_^(*3ý), za, okamžik, a-1, počasí, ...
5       [kolem-1, rozsáhlý, tlakový, výše_^(velikost_a...
6       [., v-1, místo-1_^(fyzické_umístění), s-1, níz...
7       [sobota, 10, ., listopad, 2018, studený, front...
8       [mezi-1, tlakový, níže, nad-1, východní, Atlan...
9                                                      []
10      [sobota, 10, ., listopad, 2018, kolem-1, rozsá...
11      [kolem-1, rozsáhlý, tlakový, výše_^(velikost_a...
12      [sobota, 10, ., listopad, 2018, kolem-1, rozsá...
13                                                     []
14      [kolem-1, tlakový, výše_^(velikost_apod.;_též_...
15      [zpět, teplotně_^(*1í), nadprůměrný, počasí, v...
16      [mezi-1, tlakový, níže, nad-1, východní, Atlan...
17      [sobot

In [15]:
dictionary = gensim.corpora.Dictionary(lemma_data)
dictionary.filter_extremes(no_below=20, no_above=0.35, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in lemma_data]

In [16]:
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [19]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=4, workers=4)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.015*"kuriozita" + 0.014*"záhada" + 0.014*"ikona" + 0.014*"sdílet_:T" + 0.011*"historie" + 0.004*"minuta_:B" + 0.002*"café_,t_^(kavárna)" + 0.002*"26" + 0.002*"—" + 0.002*"pražský"
Topic: 1 Word: 0.062*"5718" + 0.062*"1214" + 0.044*"ISSN_:B" + 0.007*"inzerát" + 0.006*"prodat_:W" + 0.005*"JavaScript" + 0.005*"můj_^(přivlast.)" + 0.004*"centimetr_:B" + 0.004*"autor" + 0.004*"reklama"
Topic: 2 Word: 0.009*"News-1_;K_,t_^(souč._názvů,_např._Daily_N.)" + 0.009*"Czech-2_,t" + 0.008*"centrum" + 0.007*"akciový_:B_^(jen_akciová_společnost)" + 0.006*"společnost_:B" + 0.006*"obsah" + 0.005*"souhlas" + 0.005*"šíření_^(*3it)" + 0.005*"jakýkoli" + 0.005*"server"
Topic: 3 Word: 0.007*"Patrik_;Y" + 0.006*"omezený_:B" + 0.006*"ručení_:B" + 0.005*"ISSN_:B" + 0.004*"Kurzy.cz_;S" + 0.004*"Aliaweb_;K" + 0.004*"8688" + 0.004*"1801" + 0.004*"společnost_:B" + 0.004*"Křeček_;S"
Topic: 4 Word: 0.003*""" + 0.003*"…" + 0.003*"„" + 0.003*"“" + 0.002*"?" + 0.002*"už-1" + 0.002*"člověk" + 0.002*"česk